# Error due model errors

## Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy

import dolfin_estimation as destimation

/var/folders/yy/fskfscln0pd32y_5qn562y5r0000gp/T/ipykernel_80681/2495473339.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Generate images


In [2]:
from generate_images import generate_images

generate_images(True)

*** generate_meshes ***
dim: 2
deformation_type: grav
mesh_size: 0.1
should write generate_images/square-grav-h=0.1-meshcoarse.xml
*** generate_meshes ***
dim: 2
deformation_type: grav
mesh_size: 0.05
should write generate_images/square-grav-h=0.05-meshcoarse.xml
*** generate_meshes ***
dim: 2
deformation_type: compx
mesh_size: 0.1
should write generate_images/square-compx-h=0.1-meshcoarse.xml
*** generate_meshes ***
dim: 2
deformation_type: compx
mesh_size: 0.05
should write generate_images/square-compx-h=0.05-meshcoarse.xml


## Parameters

### Varying parameters

In [2]:
mesh_size=0.1 ### default mesh size
noise_lst=[0.0, 0.1, 0.2, 0.3]
bias_lst=[0.8, 0.9, 1., 1.1, 1.2] ### studying the impact of an error of -20, -10, 0, 10, and 20 % on a model parameter on the estimation
bias_param_lst=['nu', 'b'] ### studying model errors on the Poisson ratio and on the regularization b

SNR_lst=[]
for noise in reversed(noise_lst):
    if noise==0.:
        if len(SNR_lst)!=0:
            SNR_lst.append(SNR_lst[-1]*100) ### setting the SNR arbitrarily when should be +∞
        else: ### only 0 in noise_lst
            SNR_lst.append(1000.) ### setting the SNR arbitrarily when should be +∞
    else:
        SNR_lst.append(1/noise)



### Mesh

In [3]:
cube_params = {"X0":0.2, "Y0":0.2, "X1":0.8, "Y1":0.8, "l":mesh_size}

### Material

In [4]:
nu_ref=0.3 ### defining ground-truth value of Poisson ratio
b_ref=0.3 ### defining ground-truth value of the volume regularization term
E_ref=1 #kPa

### Loading

In [5]:
load_params_body={"type":"volu", "f":0.3}

## Generating plots

In [6]:
def writing_results_to_pdf(mesh_size=0.1, SNR_lst=[], results_all={}, noise_from_images=True, regul="", bias_lst=[], bias_param="", method="EGM"):

    ### plotting parameters
    fig, ax = plt.subplots()
    plt.rc("xtick", labelsize=16)
    plt.rc("ytick", labelsize=16)
    plt.rc("legend", fontsize=12)
    plt.ylim([-100, 100])
    if bias_param=="nu":
        bias_param=r'$\nu_{truth}$'
        param_math=r'$\nu_{estim}$'
    else:
        param_math=bias_param+"$_{regul}$"
        bias_param=bias_param+"$_{truth}$"
        

    plt.xlabel("Signal to Noise Ratio (SNR)", fontsize=12)
    plt.ylabel("Estimation error (%)", fontsize=12)
    color_lst=['firebrick', 'orange', 'lawngreen', 'deepskyblue', 'orchid']

    for bias in bias_lst:
        plt.plot(SNR_lst, results_all[str(bias)]["E_average"], color=color_lst[0], label=param_math+"="+str(bias)+bias_param)
        plt.xlim([3.3, 20.])
        ax.fill_between(SNR_lst, results_all[str(bias)]["E_+"], results_all[str(bias)]["E_-"], alpha=0.5, color=color_lst[0])
        plt.gca().set_xscale('log')
        color_lst=color_lst[1:]


    ax.set_xticks([])
    ax.set_xticks([], minor=True)


    plt.legend(loc="upper right", fontsize=13, ncol=2)
    plt.grid()
    plt.savefig("./model_error_for_error_on"+str(bias_param)+"with_method="+str(method)+str(mesh_size)+"-noise_from_images="+str(noise_from_images)+"regul="+str(regul)+".pdf", bbox_inches='tight')
    plt.show()

## Collecting data

In [7]:
def run_noise_on_images(method_lst=[], load_type="body_force", load_params={}, mesh_size=0.1, cube_params={}, refine=0, SNR_lst=[], noise_lst=[], bias_param="", bias_lst=[], noise_from_images=True, regul_number=0.3):
    results_all={}
    for method in method_lst:
        for bias in bias_lst:
            results_std = {}
            results={}
            noise_results = []
            E_average, E_plus, E_minus = [], [], []
            E_all=[]
            nu_biased=nu_ref
            if bias_param=="nu":
                nu_biased=bias*nu_ref
            elif bias_param=="b":
                regul_number=bias*b_ref
            for noise in noise_lst:
                E_results=[]
                for i in range(1, 11):
                    run = str(i).zfill(2)
                    noise_results.append(noise)
                    E=destimation.identifying_parameter(method=method, nu=nu_biased, delta=1.2, load_type=load_type, load_params=load_params, mesh_size=mesh_size, cube_params=cube_params, refine=refine, noise_from_images=noise_from_images, noise=noise, regul=0.2, regul_number=regul_number, run=run)
                    E_error=(E-E_ref)/(E_ref)*100
                    E_all.append(E_error)
                    E_results.append(E_error)
                E_average.append(numpy.average(E_results))
                E_plus.append(numpy.average(E_results)+numpy.std(E_results))
                E_minus.append(numpy.average(E_results)-numpy.std(E_results))
                results_std["noise"] = noise_lst
                results_std["E_+"] = E_plus
                results_std["E_-"] = E_minus
                results_std["E_average"] = E_average
                results["noise"]=noise_results
                results["E"]=E_all
                results_all[str(bias)]=results_std
        writing_results_to_pdf(mesh_size=mesh_size, SNR_lst=SNR_lst, results_all=results_all, noise_from_images=True, method=method, regul=0.2, bias_lst=bias_lst, bias_param=bias_param)
    return
            


In [8]:
import warnings
from ffc.quadrature.deprecation \
    import QuadratureRepresentationDeprecationWarning
warnings.simplefilter("ignore", QuadratureRepresentationDeprecationWarning)

method_lst=["EGM", "VFM", "VFM_deng", "FEMU"]

results_all={}

for bias_param in bias_param_lst:
    run_noise_on_images(method_lst=method_lst, load_type="body_force", load_params=load_params_body, mesh_size=mesh_size, cube_params=cube_params, refine=False, SNR_lst=SNR_lst, bias_param=bias_param, bias_lst=bias_lst, noise_lst=noise_lst, noise_from_images=True, regul_number=b_ref)

ReadError: File run_warp_grav/square-grav-tagging-noise=0.2-run=01-h=0.1-discrete-equilibrated-tractions-normal-tangential-regul=0.2-b=0.3-refine=0_020.vtu not found.